# Trabalho 1 - Tópicos Especiais em Matemática Aplicada

**Alunos/Matricula:** João V. Farias & Renan V. Guedes / 221022604 & 221031363

**Arquitetura Usada:**

**Dataset Link:** [V1: Nonsegmented single plants (1.7GB)](https://vision.eng.au.dk/?download=/data/WeedData/Nonsegmented.zip)


### Rede Neural para determinar espécia de uma muda, dada sua imagem.

**Sobre o Dataset:** O dataset de mudas de plantas contém imagens de aproximadamente `960 plantas únicas` pertencentes a `12 espécies` em vários estágios de crescimento. Ele compreende imagens RGB anotadas com uma resolução física de aproximadamente `10 pixels por mm`.


In [3]:
print("Plant Identifier - Project 1 | TEMA")

Plant Identifier - Project 1 | TEMA


# Tratamento dos dados

- ?Redisionamento (uniformizar o tamanho das imagens)
- Normalização (valores dos pixels entre 0 e 1 `[dividir por 255]`)
- ?Data Augmentation (gerar variações das imagens `(rotações, espelhamentos, etc.)` para aumentar a diversidade do dataset)
- Separar Dataset em treino e teste

# Construção e Treinamento da NN

Para um modelo do zero, construir as camadas:

- Conv2D: Para extrair características.
- Pooling (MaxPooling2D): Para reduzir dimensionalidade.
- Dropout: Para evitar overfitting.
- Dense: Para tomar decisões baseadas nas características extraídas.

# Avaliação e Teste

- Avaliação no set de Teste
- Matriz de Confusão
- Acurácia Geral